In [2]:
from datetime import datetime
from typing import Any
import pandas as pd
import numpy as np
from math import floor
import torch
import json

from utils.table_feature_calculation import financial_feature_calculation
from tqdm.notebook import tqdm

In [3]:
with open('config/config.json', 'r') as file:
    config = json.load(file)

with open('config/config_emb_path.json', 'r') as file:
    config_emb = json.load(file)
    
rs = config['random_state']

In [4]:
year_start = '2020'

for key in config_emb.keys():
    config_emb[key] = config_emb[key].format(year_start)

# Data loading

In [5]:
df = pd.read_csv(config['ticker_data_preprocessed'], index_col=[0])
#df.drop(columns=['sector'], axis=1, inplace=True)


df = df.drop(['sector'], axis=1).T
df.index = pd.to_datetime(df.index)

if year_start == 2020 or year_start == 2021:
    df = df[(df.index < datetime(year_start, 1, 1)) ].T

In [6]:
df = pd.read_csv(config['ticker_data_preprocessed'], index_col=0)
print(df.shape)
df_pct = df.drop(['sector'], axis=1).T
df_pct.index = pd.to_datetime(df_pct.index)

tickers_list = df_pct.columns.tolist()

if year_start == 2020 or year_start == 2021:
    df_pct_train = df_pct[(df_pct.index < datetime(year_start, 1, 1)) ]
else:
    df_pct_train = df_pct

df_pct_train.tail()

(467, 1354)


,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADI,ADM,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
2023-05-12,-0.001332,-0.003554,-0.010550,-0.006791,0.003820,-0.002625,0.003998,0.018070,-0.000111,0.006408,...,-0.014125,0.003935,-0.000095,-0.002684,-0.003520,0.008519,0.000439,0.003317,-0.011023,-0.008103
2023-05-15,0.003922,0.018545,-0.007407,-0.002897,-0.003806,0.013743,-0.005883,0.001154,0.021044,0.009021,...,0.003036,-0.014226,-0.006712,0.001468,-0.001050,-0.005270,0.000512,0.017057,0.084708,0.001253
2023-05-16,-0.013282,-0.005602,-0.037802,0.000000,-0.022512,0.007269,-0.004097,0.006054,-0.008570,-0.038259,...,-0.025163,-0.019438,-0.024270,-0.012702,-0.013188,0.007258,-0.014472,-0.015959,-0.023428,-0.013054
2023-05-17,0.011719,0.050000,0.040566,0.003603,0.000419,0.001088,-0.005211,0.019485,0.028284,-0.001367,...,0.056963,-0.008259,0.022435,-0.006927,0.006682,-0.011241,-0.003782,0.019859,0.101431,-0.013833
2023-05-18,0.010722,0.009390,-0.005078,0.013666,0.000628,0.000172,-0.003216,0.010013,0.022877,-0.003559,...,0.012670,-0.002574,0.009063,0.008969,0.004618,0.008964,0.005137,0.028196,0.040886,0.004359


In [7]:
df_original = pd.read_csv(config['ticker_data_close'], index_col=0)

df_na = df_original.isna().sum()
thresh = 0.05 * len(df_original)
stocks_to_drop = df_na[df_na > thresh].index.tolist()
df_original = df_original.drop(stocks_to_drop, axis=1)
df_original = df_original.dropna(axis=0)

df_original.index = pd.to_datetime(df_original.index)
if year_start == 2020 or year_start ==  2021:
    df_original_train = df_original[(df_original.index < datetime(year_start, 1, 1)) ]
else:
    df_original_train = df_original
df_original_train.tail()

,STE,AMGN,ADSK,FCX,PTC,IRM,PHM,CHRW,PEG,BK,...,DLTR,K,NUE,SNPS,ABC,JKHY,VTR,ADM,WRK,EOG
Date,,,,,,,,,,,,,,,,,,,,,
2023-05-12,210.850006,232.949997,194.449997,34.990002,130.100006,55.090000,68.430000,99.239998,63.810001,40.150002,...,158.720001,70.809998,139.080002,368.839996,170.990005,155.809998,46.490002,75.379997,27.610001,110.419998
2023-05-15,210.830002,233.529999,196.910004,36.130001,130.779999,55.660000,68.550003,100.019997,62.549999,40.700001,...,159.149994,70.120003,140.479996,373.279999,173.339996,154.750000,47.180000,76.059998,27.860001,111.339996
2023-05-16,207.929993,227.880005,193.250000,35.250000,129.729996,54.349998,69.360001,97.970001,61.759998,39.669998,...,158.820007,69.500000,136.779999,375.140015,174.600006,147.889999,45.630001,73.150002,26.879999,107.629997
2023-05-17,205.820007,225.020004,198.460007,35.820000,131.619995,55.270000,69.500000,100.709999,61.529999,40.660000,...,158.919998,69.160004,139.639999,377.079987,174.789993,148.240005,45.840000,73.050003,27.440001,110.800003
2023-05-18,206.770004,224.229996,204.839996,35.680000,133.800003,55.169998,70.150002,101.489998,61.410000,41.080002,...,160.679993,69.050003,140.279999,409.709991,174.820007,150.699997,45.049999,72.790001,27.590000,112.019997


In [8]:
df_market = pd.read_csv(config['ticker_data_sp500'], index_col=0)
df_market.columns = ['market']
df_market.index = pd.to_datetime(df_market.index)
df_market = df_market.pct_change()[1:]
#df_market = df_market.T

if year_start == 2020 or year_start == 2021:
    df_market_train = df_market[(df_market.index < datetime(year_start, 1, 1)) ]
else:
    df_market_train = df_market.join(df_pct_train, how='inner')[['market']]

df_market_train

,market
Date,
2018-01-03,0.006399
2018-01-04,0.004029
2018-01-05,0.007034
2018-01-08,0.001662
2018-01-09,0.001303
...,...
2023-05-12,-0.001583
2023-05-15,0.002958
2023-05-16,-0.006378


# Table Data

In [9]:
df_pct_train.shape, df_market_train.shape

((1353, 467), (1353, 1))

In [10]:
df_pct_train

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADI,ADM,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
2018-01-03,0.025444,-0.012266,0.009049,-0.000174,0.015649,0.003722,0.002211,0.004615,0.012406,-0.007733,...,-0.010834,-0.006693,0.019640,-0.003426,0.012193,-0.000858,0.006932,0.019863,-0.001183,0.004598
2018-01-04,-0.007501,0.006305,0.036899,0.004645,-0.005703,-0.002225,-0.001697,0.011841,-0.001094,0.016843,...,0.005415,-0.007791,0.001384,-0.000149,0.006676,0.010180,-0.001441,0.019760,0.004147,0.005964
2018-01-05,0.015988,-0.000380,0.010631,0.011385,0.017408,0.012104,0.002890,0.008249,0.004053,-0.006675,...,0.006671,-0.007003,-0.000806,0.014051,-0.001874,0.005828,0.009941,0.015576,0.000393,0.011444
2018-01-08,0.002146,-0.009877,-0.007042,-0.003714,-0.016022,0.016576,-0.002882,0.007991,0.001745,-0.002240,...,-0.013314,0.007480,0.004496,0.006781,0.003611,0.001690,0.001905,0.009951,-0.004914,0.011996
2018-01-09,0.024554,-0.000959,-0.008080,-0.000115,0.007538,0.006398,0.001700,0.003335,-0.002069,0.003243,...,0.006778,-0.011667,-0.004246,-0.041728,0.000288,-0.002651,-0.016083,0.030643,0.023509,0.011719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-12,-0.001332,-0.003554,-0.010550,-0.006791,0.003820,-0.002625,0.003998,0.018070,-0.000111,0.006408,...,-0.014125,0.003935,-0.000095,-0.002684,-0.003520,0.008519,0.000439,0.003317,-0.011023,-0.008103
2023-05-15,0.003922,0.018545,-0.007407,-0.002897,-0.003806,0.013743,-0.005883,0.001154,0.021044,0.009021,...,0.003036,-0.014226,-0.006712,0.001468,-0.001050,-0.005270,0.000512,0.017057,0.084708,0.001253
2023-05-16,-0.013282,-0.005602,-0.037802,0.000000,-0.022512,0.007269,-0.004097,0.006054,-0.008570,-0.038259,...,-0.025163,-0.019438,-0.024270,-0.012702,-0.013188,0.007258,-0.014472,-0.015959,-0.023428,-0.013054
2023-05-17,0.011719,0.050000,0.040566,0.003603,0.000419,0.001088,-0.005211,0.019485,0.028284,-0.001367,...,0.056963,-0.008259,0.022435,-0.006927,0.006682,-0.011241,-0.003782,0.019859,0.101431,-0.013833


In [9]:
table_features_df = financial_feature_calculation(df_pct_train.T, df_market_train.T, riskless_rate=config['riskless_rate']/252)

table_features_df.to_csv(config_emb['table_finance_features'])

100%|██████████| 467/467 [00:08<00:00, 56.73it/s]


# Autoencoders

In [16]:
from autoencoders.autoencoders import Conv1dAutoEncoder, LSTMAutoEncoder, MLPAutoEncoder

In [17]:

model_mlp = MLPAutoEncoder.load_from_checkpoint(config[f'nn_mlp_checkpoint_{year_start}'],
                                            in_features=100,
                                            latent_features=100)
model_lstm = LSTMAutoEncoder.load_from_checkpoint(config[f'nn_lstm_checkpoint_{year_start}'],
                                            seq_len=100,
                                            n_features=1, 
                                            embedding_dim=100)
model_cae = Conv1dAutoEncoder.load_from_checkpoint(config[f'nn_conv_checkpoint_{year_start}'],
                                            in_channels=1,
                                            n_latent_features=100, 
                                            seq_len=100)
model_mlp.eval()
model_lstm.eval()
model_cae.eval();

1


In [19]:
mlp_encoded = np.zeros((df.shape[0], 100))
lstm_encoded = np.zeros((df.shape[0], 100))
cae_encoded = np.zeros((df.shape[0], 100))

for i, name_ticker in tqdm(enumerate(tickers_list)):
    ts_name = df_pct_train[name_ticker].values
    ts_name = ts_name.flatten()
    seq_len = ts_name.shape[0]
    fl_1 = floor(seq_len / 100)
    sample_1 = ts_name[:100 * fl_1].reshape(fl_1, 1, 100)
    fl_2 = floor(seq_len / 100)
    sample_2 = ts_name[:100 * fl_2].reshape(fl_2, 1, 100)
    
    mlp_sample = model_mlp.predict_step(torch.tensor(sample_1).float()).detach().numpy()
    cae_sample = model_cae.predict_step(torch.tensor(sample_1).float()).squeeze().detach().numpy()
    lstm_sample = model_lstm.predict_step(torch.tensor(sample_2).float()).detach().numpy()
    
    mlp_emb = mlp_sample.mean(axis=0)
    cae_emb = cae_sample.mean(axis=0)
    lstm_emb = lstm_sample.mean(axis=0)
    
    mlp_encoded[i, :] = mlp_emb
    cae_encoded[i, :] = cae_emb
    lstm_encoded[i, :] = lstm_emb

0it [00:00, ?it/s]

(13, 1, 100)
(13, 100)
(13, 1, 100)
(13, 100)
(13, 1, 100)
(13, 100)
(13, 1, 100)
(13, 100)
(13, 1, 100)
(13, 100)
(13, 1, 100)
(13, 100)
(13, 1, 100)
(13, 100)
(13, 1, 100)
(13, 100)


KeyboardInterrupt: 

In [13]:
df_mlp = pd.DataFrame(mlp_encoded, index=tickers_list)
df_cae = pd.DataFrame(cae_encoded, index=tickers_list)
df_lstm = pd.DataFrame(lstm_encoded, index=tickers_list)

df_mlp.to_csv(config_emb['autoencoder_mlp'])
df_cae.to_csv(config_emb['autoencoder_conv'])
df_lstm.to_csv(config_emb['autoencoder_lstm'])

## TSFRESH

In [14]:
from tsfresh import extract_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

from sklearn.feature_selection import SelectKBest

In [15]:
df_to_tsfresh = df_pct_train.reset_index()
df_to_tsfresh = pd.melt(df_to_tsfresh, id_vars=['index'], var_name='ticker')
df_to_tsfresh.head()

,index,ticker,value
0,2018-01-03,A,0.025444
1,2018-01-04,A,-0.007501
2,2018-01-05,A,0.015988
3,2018-01-08,A,0.002146
4,2018-01-09,A,0.024554


In [16]:
data_tsfresh = extract_features(df_to_tsfresh, column_id='ticker', n_jobs=4, column_sort='index')

features_filtered = SelectKBest(k=100).fit_transform(data_tsfresh.dropna(axis=1), df['sector'])
features_filtered  # n_instances x output_dims

Feature Extraction: 100%|██████████| 20/20 [09:11<00:00, 27.57s/it]
/cephfs/projects/psoker/.local/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [  0   1   2  11  45  46  47  48  49  50  51  52  53  54  55  56  57  58
  59  60  61  62  63  64  67  68  69  70  71  72  73  74  75  76  77  78
  79  80  81  82  83  92 105 365 670 671 672 737 738] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/cephfs/projects/psoker/.local/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


array([[0.01916639, 0.01842046, 0.01843188, ..., 0.09091864, 0.08771112,
        0.23125739],
       [0.03516995, 0.03816229, 0.03816292, ..., 0.06066927, 0.0307027 ,
        0.46494337],
       [0.02114574, 0.02140599, 0.02140857, ..., 0.08622123, 0.05736206,
        0.29595839],
       ...,
       [0.02666978, 0.02589513, 0.02591712, ..., 0.11789182, 0.10933324,
        0.21062014],
       [0.02638899, 0.02717515, 0.02717529, ..., 0.03142895, 0.03982828,
        0.21989714],
       [0.0176069 , 0.01751469, 0.01753447, ..., 0.07943728, 0.05989749,
        0.31046363]])

In [17]:
features_filtered_df = pd.DataFrame(features_filtered, index=tickers_list)
features_filtered_df.to_csv(config_emb['tsfresh'])

# TS2Vec

In [18]:
from ts2vec.ts2vec import TS2Vec

data = np.expand_dims(df_pct_train.values.T, axis=2)

# Train a TS2Vec model
model = TS2Vec(
    input_dims=1,
    device=0,
    output_dims=100
)
loss_log = model.fit(
    data,
    verbose=True
)

emb_ts2vec = model.encode(data, encoding_window='full_series') 
ts2vec_df = pd.DataFrame(emb_ts2vec, index=tickers_list)
ts2vec_df.to_csv(config_emb['ts2vec'])

Epoch #0: loss=3.594549631250316
Epoch #1: loss=3.6127338573850434
Epoch #2: loss=3.5883763000882904
Epoch #3: loss=3.584869220339019
Epoch #4: loss=3.5758077276164086
Epoch #5: loss=3.6251190941909264
Epoch #6: loss=3.511380877988092
Epoch #7: loss=3.5761959634978195
Epoch #8: loss=3.5097599029541016
Epoch #9: loss=3.4866890578434386
Epoch #10: loss=3.574826585835424
Epoch #11: loss=3.4940706368150383
Epoch #12: loss=3.528484426695725
Epoch #13: loss=3.5048284201786437
Epoch #14: loss=3.6102628461245834
Epoch #15: loss=3.4311317904242156
Epoch #16: loss=3.4773950165715712
Epoch #17: loss=3.5335126416436555
Epoch #18: loss=3.5361071290640997
Epoch #19: loss=3.465371173003624


## PCA and FastICA

In [12]:
from sklearn.decomposition import PCA, FastICA
from sklearn.manifold import TSNE
from umap import UMAP

In [20]:
df_pct_train.shape

(1353, 467)

In [13]:
pca_data = PCA(n_components=100, random_state=rs).fit_transform(df_pct_train.T)
fastica_data =  FastICA(n_components=100, random_state=rs).fit_transform(df_pct_train.T)
tnse_data = TSNE(n_components=3, random_state=rs).fit_transform(df_pct_train.T)
umap_data = UMAP(n_components=100, random_state=rs).fit_transform(df_pct_train.T)

/cephfs/projects/psoker/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['Timestamp']. An error will be raised in 1.2.
  warnings.warn(
/cephfs/projects/psoker/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['Timestamp']. An error will be raised in 1.2.
  warnings.warn(
/cephfs/projects/psoker/.local/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/cephfs/projects/psoker/.local/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
/cephfs/projects/psoker/.local/lib/python3.8/site-packages/sklearn/utils/validation.p

In [14]:
pca_data

array([[-1.64430651e-01,  1.44510007e-01,  6.67265681e-02, ...,
         2.25993736e-03,  1.49710331e-02,  1.38785063e-02],
       [ 6.20504884e-01,  3.72489057e-02, -3.60873587e-01, ...,
        -1.34421936e-02, -1.05414498e-02,  5.99945496e-04],
       [-6.49586462e-02, -3.14313680e-02, -1.05077686e-01, ...,
         1.63963529e-02,  2.57610259e-02, -1.33988606e-02],
       ...,
       [ 7.37800615e-04,  2.74966999e-01,  4.92457439e-02, ...,
         1.79459609e-02,  4.44605356e-02,  2.87308126e-04],
       [ 3.49160818e-01, -1.94425082e-01, -5.57978310e-02, ...,
        -4.06439719e-03,  3.14718262e-02, -4.50313181e-02],
       [-2.22832339e-01,  1.03350958e-01,  5.38730057e-03, ...,
        -2.32746251e-02,  2.60281313e-02,  1.79293052e-02]])

In [ ]:
pca_df = pd.DataFrame(pca_data, index=tickers_list)
fastica_df = pd.DataFrame(fastica_data, index=tickers_list)
tnse_df =  pd.DataFrame(tnse_data, index=tickers_list)
umap_df =  pd.DataFrame(umap_data, index=tickers_list)

pca_df.to_csv(config_emb['pca'])
fastica_df.to_csv(config_emb['fastica'])
tnse_df.to_csv(config_emb['tnse'])
umap_df.to_csv(config_emb['umap'])